In [1]:
%matplotlib inline

from urllib.request import urlretrieve

import matplotlib.pyplot as plt
import nengo
from nengo import spa
import numpy as np
import tensorflow as tf

import nengo_dl

from scipy.signal import decimate

In [3]:
from src.io.psee_loader import PSEELoader

import pathlib
from glob import glob

In [4]:
delta_t = 10000
n = 75
skip = 100

In [19]:
vids = map(PSEELoader, glob("Prophesee_Dataset_n_cars/n-cars_train/cars/*_td.dat"))
# height, width = vid.get_size()
height, width = 120, 120
# vid.seek_time(skip)
event_seqs = [next(vids).load_delta_t(delta_t) for _ in range(n)]
# N = len(event_seqs)

print(event_seqs[0])

[(   0, 48, 13, 1) (  74, 27, 42, 1) ( 102, 19, 12, 0) ( 111, 30, 43, 1)
 ( 112, 16, 17, 1) ( 127, 23,  6, 0) ( 138, 53,  8, 0) ( 168, 23, 10, 0)
 ( 171, 13,  7, 1) ( 197, 21,  6, 1) ( 297, 28, 43, 1) ( 319,  5, 38, 1)
 ( 348, 23, 17, 0) ( 421, 59, 14, 0) ( 450,  5, 40, 1) ( 476,  8, 42, 0)
 ( 477, 19, 10, 0) ( 492, 20, 22, 0) ( 492, 19,  9, 0) ( 493, 29, 21, 0)
 ( 533, 23,  7, 0) ( 545, 14,  6, 0) ( 577,  0, 26, 0) ( 606,  9, 44, 0)
 ( 621, 18, 22, 1) ( 632,  4,  3, 1) ( 632,  0,  3, 1) ( 644,  1, 24, 0)
 ( 650, 35, 20, 1) ( 707, 29, 43, 1) ( 737,  8, 40, 0) ( 740, 51, 30, 1)
 ( 810, 58, 16, 0) ( 810, 46, 20, 0) ( 887, 28, 37, 0) ( 900, 21,  5, 1)
 ( 960, 54, 26, 1) ( 966,  3, 44, 1) ( 990, 13, 15, 0) (1002, 56, 13, 1)
 (1091,  8, 41, 0) (1099,  8,  9, 1) (1118, 22,  6, 1) (1154,  1, 21, 0)
 (1189,  6, 40, 1) (1190, 48, 11, 1) (1195, 23,  6, 0) (1254,  2, 17, 1)
 (1257,  8, 12, 1) (1258, 48, 12, 1) (1343,  4,  7, 1) (1381, 32,  0, 1)
 (1452,  8, 16, 1) (1487, 43, 43, 1) (1525, 20, 14,

In [20]:
dat = np.zeros((n, delta_t, 30, 30),dtype=np.int8)
for i,block in enumerate(event_seqs):
    for ev in block:
        dat[i,ev[0],ev[1]//4,ev[2]//4] = ev[3]
# for ev in events:
#     dat[ev[0],ev[1],ev[2]] = ev[3]

# dat = decimate(dat,4,axis=2)
# dat = decimate(dat,4,axis=3)

train_data = dat[0:60,:,:,:]
test_data  = dat[60:75,:,:,:]

train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], -1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], -1))

print(train_data.shape, test_data.shape)

(60, 10000, 900) (15, 10000, 900)


In [23]:
with nengo.Network() as auto_net:
    n_in = train_data.shape[2]
    n_hidden=64
    neuron_type = nengo.LIF(amplitude=0.01)
    
    # input
    nengo_a = nengo.Node(np.zeros(n_in))

    # first layer
    nengo_b = nengo.Ensemble(n_hidden, 1, neuron_type=neuron_type)
    nengo.Connection(nengo_a, nengo_b.neurons, transform=nengo_dl.dists.Glorot())

    # second layer
    nengo_c = nengo.Ensemble(n_in, 1, neuron_type=neuron_type)
    nengo.Connection(
        nengo_b.neurons, nengo_c.neurons, transform=nengo_dl.dists.Glorot()
    )

    # probes are used to collect data from the network
    p_c = nengo.Probe(nengo_c.neurons)
    p_b = nengo.Probe(nengo_b.neurons)

In [22]:
minibatch_size = 8

with nengo_dl.Simulator(network=auto_net, minibatch_size=minibatch_size) as sim:
    out = sim.predict(np.ones((minibatch_size, 1, n_in)))
    print(out[p_c].shape,out[p_b].shape)

|                     Building network (0%)                    | ETA:  --:--:--
Build finished in 0:00:00
|#                         Optimizing graph                           | 0:00:00
|#             Optimizing graph: operator simplificaton               | 0:00:00
Optimizing graph: operator simplificaton finished in 0:00:00
|#                Optimizing graph: merging operators                 | 0:00:00
Optimizing graph: merging operators finished in 0:00:00
|#                Optimizing graph: ordering signals                  | 0:00:00
Optimizing graph: ordering signals finished in 0:00:00
|#                Optimizing graph: creating signals                  | 0:00:00
Optimizing graph: creating signals finished in 0:00:00
Optimization finished in 0:00:00
|#                        Constructing graph                          | 0:00:00
|               #         Constructing graph                          | 0:00:01
|           Constructing graph: pre-build stage (0%)           | ETA:  --:

/home/karthik/sim_clean/nengo-dl/nengo_dl/simulator.py:1024: UserWarning: Running for one timestep, but the network contains synaptic filters (which will introduce at least a one-timestep delay); did you mean to set synapse=None?
  warnings.warn(


|############Constructing graph: build stage (100%)############| ETA:  00:00:00
Constructing graph: build stage finished in 0:00:00


2022-03-06 15:44:29.294703: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-03-06 15:44:29.294751: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:438 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support
2022-03-06 15:44:29.298357: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-03-06 15:44:29.298386: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:438 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: 2 root error(s) found.
  (0) INTERNAL:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node keras_model/TensorGraph/while/iteration_0/DotIncBuilder/MatMul
 (defined at /home/karthik/sim_clean/nengo-dl/nengo_dl/op_builders.py:348)
]]
	 [[keras_model/TensorGraph/while/exit/_130/_8]]
  (1) INTERNAL:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node keras_model/TensorGraph/while/iteration_0/DotIncBuilder/MatMul
 (defined at /home/karthik/sim_clean/nengo-dl/nengo_dl/op_builders.py:348)
]]
0 successful operations.
0 derived errors ignored. [Op:__inference_predict_function_918]

Errors may have originated from an input operation.
Input Source operations connected to node keras_model/TensorGraph/while/iteration_0/DotIncBuilder/MatMul:
In[0] keras_model/TensorGraph/while/iteration_0/DotIncBuilder/Reshape (defined at /home/karthik/sim_clean/nengo-dl/nengo_dl/signals.py:420)	
In[1] keras_model/TensorGraph/wh